<h1> On Gradients with Effective-Quadratures

In [1]:
%matplotlib inline
from effective_quadratures.parameter import Parameter
from effective_quadratures.polynomial import Polynomial
from effective_quadratures.indexset import IndexSet
from effective_quadratures.computestats import Statistics
from effective_quadratures.effectivequads import EffectiveSubsampling
from effective_quadratures.utils import meshgrid, twoDgrid, evalfunction, evalgradients
import numpy as np
import matplotlib.pyplot as plt

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Consider the following function and its gradient:
$$f(x_1, x_2) = cos(x_1) + sin(2x_2),$$
$$\nabla f=\left(\begin{array}{c}
-sin(x_{1})\\
2cos(2x_{2})
\end{array}\right).$$

We start by defining our computational model:

In [2]:
def fun(x):
    return np.cos(x[0]) + np.sin(2*x[1])

def fungrad(x):
    return [-np.sin(x[0]),2*np.cos(2*x[1])] 

In [3]:
x1 = Parameter(param_type="Uniform", lower=-1, upper=1, points=3, derivative_flag=1)
x2 = Parameter(param_type="Uniform", lower=-1, upper=1, points=3, derivative_flag=1)
parameters = [x1, x2]

Now select a hyperbolic basis

In [4]:
hyperbolic_cross = IndexSet("Hyperbolic basis", orders=[2,2], q=1.0)
maximum_number_of_evals = hyperbolic_cross.getCardinality()

Some useful text goes here...

In [5]:
esq = EffectiveSubsampling(parameters, hyperbolic_cross)
points = esq.getEffectivelySubsampledPoints(maximum_number_of_evals+5)
f = evalfunction(points, fun)
df = evalgradients(points, fungrad)
x = esq.solveLeastSquaresWithGradients(maximum_number_of_evals+5, f, df)

12 18
9 6
ERROR: mismatch in sizes of C and d


SystemExit: 

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
x = esq.solveLeastSquares(maximum_number_of_evals+5, f)
print x